In [11]:
import pandas as pd
np=pd.np
from sdd_api.api import Api
from credentials import *
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns=None
api = Api(username=username, password=password, client_id=client_id, client_secret=client_secret)

In [18]:
matchups=api.get_dataframe("matchups",season_start=2000)
matchups

,season,home_name,away_name,full_game_type,away_first_downs,away_pass_yds,away_pts,away_rush_yds,away_to,away_yards,game_datetime,game_location,game_outcome,game_time,game_type,had_overtime,home_first_downs,home_pass_yds,home_pts,home_rush_yds,home_to,home_yards,teams,week_num,away_conference,conference_play,home_line,over_under,handicap_difference,home_conference,line_moves,matchup_id
0,2000,KC,IND,Week 1,20.0,267.0,27.0,119.0,2.0,386.0,967986000000,None,L,1:00PM ET,Regular,False,17.0,206.0,14.0,74.0,1.0,280.0,"IND,KC",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903IND@KC
1,2000,MIN,CHI,Week 1,23.0,272.0,27.0,153.0,1.0,425.0,967986000000,None,W,1:00PM ET,Regular,False,17.0,188.0,30.0,186.0,1.0,374.0,"CHI,MIN",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903CHI@MIN
2,2000,WAS,CAR,Week 1,12.0,124.0,17.0,112.0,1.0,236.0,967986060000,None,W,1:01PM ET,Regular,False,23.0,234.0,20.0,162.0,NaN,396.0,"CAR,WAS",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903CAR@WAS
3,2000,ATL,SF,Week 1,23.0,247.0,28.0,92.0,1.0,339.0,967986120000,None,W,1:02PM ET,Regular,False,22.0,264.0,36.0,95.0,1.0,359.0,"ATL,SF",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903SF@ATL
4,2000,CLE,JAC,Week 1,28.0,279.0,27.0,119.0,NaN,398.0,967986120000,None,L,1:02PM ET,Regular,False,9.0,153.0,7.0,96.0,1.0,249.0,"CLE,JAC",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903JAC@CLE
5,2000,NE,TB,Week 1,17.0,156.0,21.0,140.0,1.0,296.0,967986120000,None,L,1:02PM ET,Regular,False,14.0,190.0,16.0,88.0,1.0,278.0,"NE,TB",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903TB@NE
6,2000,PIT,BAL,Week 1,18.0,196.0,16.0,140.0,NaN,336.0,967986120000,None,L,1:02PM ET,Regular,False,12.0,193.0,0.0,30.0,1.0,223.0,"BAL,PIT",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903BAL@PIT
7,2000,NO,DET,Week 1,10.0,89.0,14.0,98.0,2.0,187.0,967986180000,None,L,1:03PM ET,Regular,False,13.0,146.0,10.0,106.0,3.0,252.0,"DET,NO",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903DET@NO
8,2000,NYG,ARI,Week 1,19.0,312.0,16.0,43.0,4.0,355.0,967986240000,None,W,1:04PM ET,Regular,False,20.0,172.0,21.0,223.0,2.0,395.0,"ARI,NYG",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903ARI@NYG
9,2000,DAL,PHI,Week 1,27.0,119.0,41.0,306.0,3.0,425.0,967997100000,None,L,4:05PM ET,Regular,False,10.0,100.0,14.0,67.0,2.0,167.0,"DAL,PHI",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903PHI@DAL


### Lets Predict the Game Winner  
We'll use data from 2011 season and on since we have line information for these games

In [32]:
matchups=matchups[(matchups['game_type'].isin(['Regular','Playoffs']))]
#drop any unscored/unplayed games
matchups=matchups.dropna(subset=["home_pts","away_pts"])
matchups.tail(2)

,season,home_name,away_name,full_game_type,away_first_downs,away_pass_yds,away_pts,away_rush_yds,away_to,away_yards,game_datetime,game_location,game_outcome,game_time,game_type,had_overtime,home_first_downs,home_pass_yds,home_pts,home_rush_yds,home_to,home_yards,teams,week_num,away_conference,conference_play,home_line,over_under,handicap_difference,home_conference,line_moves,matchup_id
4542,2017,MIN,NO,Week 1,19.0,284.0,19.0,60.0,NaN,344.0,1505157000000,,W,7:10PM ET,Regular,False,23.0,341.0,29.0,129.0,NaN,470.0,"MIN,NO",1.0,NFC,NFC,-3.5,48.0,0.0,NFC,NaN,20170911NO@MIN
4543,2017,DEN,LAC,Week 1,17.0,185.0,21.0,64.0,1.0,249.0,1505168400000,,W,10:20PM ET,Regular,False,22.0,181.0,24.0,140.0,2.0,321.0,"DEN,LAC",1.0,AFC,AFC,-3.5,43.5,1.0,AFC,NaN,20170911LAC@DEN


In [33]:
matchups.describe()

,season,away_first_downs,away_pass_yds,away_pts,away_rush_yds,away_to,away_yards,game_datetime,home_first_downs,home_pass_yds,home_pts,home_rush_yds,home_to,home_yards,week_num,home_line,over_under,handicap_difference,line_moves
count,4538.000000,4538.000000,4536.000000,4538.000000,4538.000000,3617.000000,4538.000000,4.538000e+03,4538.000000,4538.000000,4538.000000,4538.000000,3536.000000,4538.000000,4538.000000,1614.000000,1614.000000,1614.000000,1599.000000
mean,2008.056192,18.526443,216.604056,20.501763,109.404363,2.138789,325.912957,1.227917e+12,19.576906,223.095637,23.105773,118.054429,2.097568,341.150066,8.793081,-2.280669,44.090768,0.472119,20.479049
std,4.906727,5.027527,78.550309,10.084594,50.876863,1.209788,86.211580,1.548654e+11,4.975976,77.142130,10.379160,52.409868,1.193966,83.922011,5.099035,5.585516,8.185730,1.057183,19.106006
min,2000.000000,2.000000,-9.000000,0.000000,-18.000000,1.000000,26.000000,9.679860e+11,3.000000,6.000000,0.000000,-3.000000,1.000000,77.000000,1.000000,-26.500000,0.000000,0.000000,0.000000
25%,2004.000000,15.000000,160.000000,13.000000,72.000000,1.000000,266.250000,1.098018e+12,16.000000,169.000000,16.000000,81.000000,1.000000,283.000000,4.000000,-6.500000,42.000000,0.000000,14.000000
50%,2008.000000,19.000000,213.000000,20.000000,102.000000,2.000000,326.000000,1.226841e+12,19.000000,218.000000,23.000000,112.000000,2.000000,339.500000,9.000000,-3.000000,45.000000,0.000000,19.000000
75%,2012.000000,22.000000,269.000000,27.000000,139.000000,3.000000,386.000000,1.355663e+12,23.000000,272.000000,30.000000,148.000000,3.000000,396.000000,13.000000,2.500000,48.000000,0.500000,25.000000
max,2017.000000,36.000000,516.000000,59.000000,351.000000,8.000000,626.000000,1.505168e+12,40.000000,522.000000,62.000000,407.000000,8.000000,653.000000,17.000000,16.000000,60.000000,9.000000,673.000000


### We will use 538's Elo Algorithm
- [538's Elo Introduction](https://fivethirtyeight.com/datalab/introducing-nfl-elo-ratings/)
- [More Elo Description](https://fivethirtyeight.com/datalab/nfl-elo-ratings-are-back/)  

### The features for this model are:
- game location (home, away, neutral)
- team score
- opponent score

In [34]:
matchups['is_neutral']=matchups['game_location'].apply(lambda x: True if x=="N" else False)

In [35]:
from collections import defaultdict
def silverK(MOV, elo_diff):
    K_0=20
    multiplier=np.log(abs(MOV)+1)*(2.2/((elo_diff)*.001+2.2))
    return K_0*multiplier,K_0*multiplier

def silverS(home_score, away_score):
    S_home,S_away=0,0
    if home_score>away_score:
        S_home=1
    elif away_score>home_score:
        S_away=1
    else:
        S_home,S_away=.5,.5
    return S_home,S_away

def silver_elo_update(home_score, away_score, home_rating, away_rating, isNeutral):
    HOME_AD=65.
    if not isNeutral:
        pass#home_rating+=HOME_AD
    E_home = elo_prediction(home_rating,away_rating)
    E_away=1-E_home
    elo_diff=home_rating-away_rating
    MOV=home_score-away_score
    
    S_home,S_away = silverS(home_score,away_score)
    if home_score>=away_score:
        elo_winner=home_rating
        elo_loser=away_rating
    else:
        elo_loser=home_rating
        elo_winner=away_rating

    K_home,K_away =  silverK(MOV,elo_winner-elo_loser)
        
    return K_home*(S_home-E_home),K_away*(S_away-E_away)

def elo_prediction(home_rating,away_rating):
    E_home = 1./(1 + 10 ** ((away_rating - home_rating) / (400.)))
    return E_home

def score_prediction(home_rating,away_rating):
    return (home_rating-away_rating)/25.
class HeadToHeadModel(object):
    def __init__(self, events, update_function, prediction_function=None):
        self.update_function=update_function
        self.events=events
        self.ratings=defaultdict(lambda: 1505)
        self.prediction_function = prediction_function
        self.predictions = []
        self.curr_season=defaultdict(lambda: self.events[0][1]['season'])

    def compute_elo_ratings(self):
        for idx, event in self.events:
            new_year=event['season']
            label_i=event['home_name']
            label_j=event['away_name']
            
            if self.curr_season[label_i]!=new_year:
                self.curr_season[label_i]=new_year
                self.ratings[label_i]=self.ratings[label_i]*1/3+1505.*2/3
            elif self.curr_season[label_j]!=new_year:
                self.curr_season[label_j]=new_year
                self.ratings[label_j]=self.ratings[label_j]*.75+1505.*.25
            #todo change below to just use event
            update=self.update_function(event['home_pts'],event['away_pts'], self.ratings[label_i], self.ratings[label_j], event['is_neutral'])
            self.ratings[label_i]+=update[0]
            self.ratings[label_j]+=update[1]
            

    def power_rankings(self):
        from operator import itemgetter
        #sort dictionary by value to get ascending list of teams
        power_rankings = sorted(self.ratings.items(), key=itemgetter(1), reverse=True)
        power = []
        #Make the 0-th team 1st
        for i, x in enumerate(power_rankings):
            power.append((i + 1, x))
        return power
m=HeadToHeadModel(list(matchups.iterrows()), silver_elo_update, elo_prediction)
m.compute_elo_ratings()
m.power_rankings()

[(1, ('ATL', 1681.6698906645811)),
 (2, ('KC', 1666.2094134864094)),
 (3, ('PIT', 1629.5527688127345)),
 (4, ('DAL', 1569.6434861959603)),
 (5, ('NE', 1567.3878243723489)),
 (6, ('GB', 1566.8958832392466)),
 (7, ('OAK', 1561.4944668786957)),
 (8, ('SEA', 1550.3393609641466)),
 (9, ('PHI', 1548.8475800467454)),
 (10, ('CAR', 1539.5779269427005)),
 (11, ('DEN', 1533.3809497755301)),
 (12, ('DET', 1525.6346830480454)),
 (13, ('MIN', 1524.9337987241013)),
 (14, ('BAL', 1524.2491803750747)),
 (15, ('MIA', 1517.6222852397475)),
 (16, ('NYG', 1512.4250190710479)),
 (17, ('BUF', 1512.1435544505125)),
 (18, ('TB', 1508.1235602250397)),
 (19, ('LAR', 1499.2007671461131)),
 (20, ('ARI', 1494.7377965485427)),
 (21, ('WAS', 1483.7868728591238)),
 (22, ('NO', 1480.0021242062619)),
 (23, ('CIN', 1477.3360723367971)),
 (24, ('TEN', 1476.2020945470833)),
 (25, ('IND', 1474.3959004314734)),
 (26, ('HOU', 1456.9139883030996)),
 (27, ('CHI', 1444.2231515036867)),
 (28, ('CLE', 1427.6506211023132)),
 (29, 

### What Good are Rankings without Testing?
Elo is a robust algorithm for strength of schedule based ratings. But what good are rankings without testing them?!?. Let's test our rankings at picking the winners and against the spread.

In [36]:
#Let's redo our elo model with the addition of elo ranking output
class HeadToHeadModel(object):
    def __init__(self, events, update_function, prediction_function=None):
        self.update_function=update_function
        self.events=events
        self.ratings=defaultdict(lambda: 1505)
        self.prediction_function = prediction_function
        self.predictions = []
        self.curr_season=defaultdict(lambda: self.events[0][1]['season'])
        self.elos=[]
    def compute_elo_ratings(self):
        for idx, event in self.events:
            new_year=event['season']
            label_i=event['home_name']
            label_j=event['away_name']
            
            if self.curr_season[label_i]!=new_year:
                self.curr_season[label_i]=new_year
                self.ratings[label_i]=self.ratings[label_i]*1/3+1505.*2/3
            elif self.curr_season[label_j]!=new_year:
                self.curr_season[label_j]=new_year
                self.ratings[label_j]=self.ratings[label_j]*.75+1505.*.25
            #todo change below to just use event
            update=self.update_function(event['home_pts'],event['away_pts'], self.ratings[label_i], self.ratings[label_j], event['is_neutral'])
            self.elos.append({
                "home_elo":self.ratings[label_i],
                "away_elo":self.ratings[label_j],
                "index": idx,
                
                             })
            self.ratings[label_i]+=update[0]
            self.ratings[label_j]+=update[1]
            

            

    def power_rankings(self):
        from operator import itemgetter
        #sort dictionary by value to get ascending list of teams
        power_rankings = sorted(self.ratings.items(), key=itemgetter(1), reverse=True)
        power = []
        #Make the 0-th team 1st
        for i, x in enumerate(power_rankings):
            power.append((i + 1, x))
        return power
m=HeadToHeadModel(list(matchups.iterrows()), silver_elo_update, elo_prediction)
m.compute_elo_ratings()
m.power_rankings()

[(1, ('ATL', 1681.6698906645811)),
 (2, ('KC', 1666.2094134864094)),
 (3, ('PIT', 1629.5527688127345)),
 (4, ('DAL', 1569.6434861959603)),
 (5, ('NE', 1567.3878243723489)),
 (6, ('GB', 1566.8958832392466)),
 (7, ('OAK', 1561.4944668786957)),
 (8, ('SEA', 1550.3393609641466)),
 (9, ('PHI', 1548.8475800467454)),
 (10, ('CAR', 1539.5779269427005)),
 (11, ('DEN', 1533.3809497755301)),
 (12, ('DET', 1525.6346830480454)),
 (13, ('MIN', 1524.9337987241013)),
 (14, ('BAL', 1524.2491803750747)),
 (15, ('MIA', 1517.6222852397475)),
 (16, ('NYG', 1512.4250190710479)),
 (17, ('BUF', 1512.1435544505125)),
 (18, ('TB', 1508.1235602250397)),
 (19, ('LAR', 1499.2007671461131)),
 (20, ('ARI', 1494.7377965485427)),
 (21, ('WAS', 1483.7868728591238)),
 (22, ('NO', 1480.0021242062619)),
 (23, ('CIN', 1477.3360723367971)),
 (24, ('TEN', 1476.2020945470833)),
 (25, ('IND', 1474.3959004314734)),
 (26, ('HOU', 1456.9139883030996)),
 (27, ('CHI', 1444.2231515036867)),
 (28, ('CLE', 1427.6506211023132)),
 (29, 

In [37]:
elo=pd.DataFrame(m.elos).set_index("index").join(matchups)
elo.head(2)

,away_elo,home_elo,season,home_name,away_name,full_game_type,away_first_downs,away_pass_yds,away_pts,away_rush_yds,away_to,away_yards,game_datetime,game_location,game_outcome,game_time,game_type,had_overtime,home_first_downs,home_pass_yds,home_pts,home_rush_yds,home_to,home_yards,teams,week_num,away_conference,conference_play,home_line,over_under,handicap_difference,home_conference,line_moves,matchup_id,is_neutral
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1505.0,1505.0,2000,KC,IND,Week 1,20.0,267.0,27.0,119.0,2.0,386.0,967986000000,None,L,1:00PM ET,Regular,False,17.0,206.0,14.0,74.0,1.0,280.0,"IND,KC",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903IND@KC,False
1,1505.0,1505.0,2000,MIN,CHI,Week 1,23.0,272.0,27.0,153.0,1.0,425.0,967986000000,None,W,1:00PM ET,Regular,False,17.0,188.0,30.0,186.0,1.0,374.0,"CHI,MIN",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903CHI@MIN,False


In [38]:
elo.describe()

,away_elo,home_elo,season,away_first_downs,away_pass_yds,away_pts,away_rush_yds,away_to,away_yards,game_datetime,home_first_downs,home_pass_yds,home_pts,home_rush_yds,home_to,home_yards,week_num,home_line,over_under,handicap_difference,line_moves
count,4538.000000,4538.000000,4538.000000,4538.000000,4536.000000,4538.000000,4538.000000,3617.000000,4538.000000,4.538000e+03,4538.000000,4538.000000,4538.000000,4538.000000,3536.000000,4538.000000,4538.000000,1614.000000,1614.000000,1614.000000,1599.000000
mean,1513.673080,1510.808996,2008.056192,18.526443,216.604056,20.501763,109.404363,2.138789,325.912957,1.227917e+12,19.576906,223.095637,23.105773,118.054429,2.097568,341.150066,8.793081,-2.280669,44.090768,0.472119,20.479049
std,81.186389,79.228516,4.906727,5.027527,78.550309,10.084594,50.876863,1.209788,86.211580,1.548654e+11,4.975976,77.142130,10.379160,52.409868,1.193966,83.922011,5.099035,5.585516,8.185730,1.057183,19.106006
min,1265.552531,1273.134241,2000.000000,2.000000,-9.000000,0.000000,-18.000000,1.000000,26.000000,9.679860e+11,3.000000,6.000000,0.000000,-3.000000,1.000000,77.000000,1.000000,-26.500000,0.000000,0.000000,0.000000
25%,1459.174796,1459.846843,2004.000000,15.000000,160.000000,13.000000,72.000000,1.000000,266.250000,1.098018e+12,16.000000,169.000000,16.000000,81.000000,1.000000,283.000000,4.000000,-6.500000,42.000000,0.000000,14.000000
50%,1513.771517,1509.568994,2008.000000,19.000000,213.000000,20.000000,102.000000,2.000000,326.000000,1.226841e+12,19.000000,218.000000,23.000000,112.000000,2.000000,339.500000,9.000000,-3.000000,45.000000,0.000000,19.000000
75%,1568.566008,1560.887391,2012.000000,22.000000,269.000000,27.000000,139.000000,3.000000,386.000000,1.355663e+12,23.000000,272.000000,30.000000,148.000000,3.000000,396.000000,13.000000,2.500000,48.000000,0.500000,25.000000
max,1812.268800,1798.012452,2017.000000,36.000000,516.000000,59.000000,351.000000,8.000000,626.000000,1.505168e+12,40.000000,522.000000,62.000000,407.000000,8.000000,653.000000,17.000000,16.000000,60.000000,9.000000,673.000000


### Prediction and Testing
Since we are seeding our rankings starting with the 2011 season it would be unfair to our model to expect to it be right in the beginning. For our purposes, we'll look at performance during the whole season and during weeks 4-15 to allow rating to settle and not lose when a team rests their players in weeks 16 and 17

In [39]:
def predict_home_margin(row):
    #how many points the home team is expected to win bye
    return score_prediction(row['home_elo']+65, row['away_elo'])
elo['predicted_home_margin']=elo.apply(predict_home_margin,axis=1)

In [40]:
elo.head(1)

,away_elo,home_elo,season,home_name,away_name,full_game_type,away_first_downs,away_pass_yds,away_pts,away_rush_yds,away_to,away_yards,game_datetime,game_location,game_outcome,game_time,game_type,had_overtime,home_first_downs,home_pass_yds,home_pts,home_rush_yds,home_to,home_yards,teams,week_num,away_conference,conference_play,home_line,over_under,handicap_difference,home_conference,line_moves,matchup_id,is_neutral,predicted_home_margin
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1505.0,1505.0,2000,KC,IND,Week 1,20.0,267.0,27.0,119.0,2.0,386.0,967986000000,None,L,1:00PM ET,Regular,False,17.0,206.0,14.0,74.0,1.0,280.0,"IND,KC",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903IND@KC,False,2.6


In [41]:
#1 is home winning
elo['predicted_winner']=elo.apply(lambda row: 1 if row['predicted_home_margin']>=0 else 0, axis=1)


In [42]:
elo['vegas_predicted_winner']=elo.apply(lambda row: 0 if row['home_line']>0 else 1, axis=1)#line assigns negative value
elo['winner']=elo.apply(lambda row: 1 if row['home_pts']>row['away_pts'] else 0, axis=1)#does not account for ties

Since this is a binary decision problem, either win or lose our bet, we will use [confusion matrices](https://en.wikipedia.org/wiki/Confusion_matrix) to judge the performance of our model

In [43]:
from sklearn.metrics import confusion_matrix
def print_confusion_matrix(y_true, y_pred):
    conf_matrix=confusion_matrix(y_true, y_pred)
    success_rate=np.trace(conf_matrix)/np.sum(conf_matrix)
    print(success_rate)
    print(conf_matrix)
test_period=elo[(elo['week_num'].between(4,15,inclusive=True))&(elo['game_type']=='Regular')&(elo['season'].between(2012,2016))]

In [44]:
y_true='winner'
y_pred='predicted_winner'
print_confusion_matrix(test_period['winner'],test_period['predicted_winner'])

0.644318181818
[[168 223]
 [ 90 399]]


In [45]:
y_true='winner'
y_pred='vegas_predicted_winner'
print_confusion_matrix(test_period['winner'],test_period[y_pred])

0.659090909091
[[200 191]
 [109 380]]


### Vegas is pretty good huh
We were good at predicting winners but vegas is slightly better... The highest we've gotten with a pure elo approach using more years of training is 67%. Let's look at our against the spread performance and see if we can find an edge.

In [46]:
elo.head(1)

,away_elo,home_elo,season,home_name,away_name,full_game_type,away_first_downs,away_pass_yds,away_pts,away_rush_yds,away_to,away_yards,game_datetime,game_location,game_outcome,game_time,game_type,had_overtime,home_first_downs,home_pass_yds,home_pts,home_rush_yds,home_to,home_yards,teams,week_num,away_conference,conference_play,home_line,over_under,handicap_difference,home_conference,line_moves,matchup_id,is_neutral,predicted_home_margin,predicted_winner,vegas_predicted_winner,winner
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1505.0,1505.0,2000,KC,IND,Week 1,20.0,267.0,27.0,119.0,2.0,386.0,967986000000,None,L,1:00PM ET,Regular,False,17.0,206.0,14.0,74.0,1.0,280.0,"IND,KC",1.0,None,None,NaN,NaN,NaN,None,NaN,20000903IND@KC,False,2.6,1,1,0


In [47]:
elo['home_margin']=elo.apply(lambda x: x['home_pts']-x['away_pts'], axis=1)
elo['home_bet']=elo.apply(lambda x: (x['predicted_home_margin']+x['home_line'])<0,axis=1)
elo['home_covers']=elo.apply(lambda x: (x['home_margin']+x['home_line'])>0,axis=1)
test_period=elo[(elo['week_num'].between(4,15,inclusive=True))&(elo['game_type']=='Regular')&(elo['season'].between(2012,2016))]
y_true='home_covers'
y_pred='home_bet'
print_confusion_matrix(test_period[y_true],test_period[y_pred])

0.501136363636
[[248 224]
 [215 193]]


That won't win for us. Under standard wagering strategies you need to be better than 52.4%

### Performance each season during test period

In [48]:
elo['predicted_winner_right']=elo.apply(lambda x: 1 if x['winner']==x['predicted_winner'] else 0, axis=1)
elo['vegas_favored_wins']=elo.apply(lambda x: 1 if x['winner']==x['vegas_predicted_winner'] else 0, axis=1)
elo['ats_right']=elo.apply(lambda x: 1 if x['home_covers']==x['home_bet'] else 0, axis=1)
test_period=elo[(elo['week_num'].between(4,15,inclusive=True))&(elo['game_type']=='Regular')&(elo['season'].between(2012,2016))]


In [49]:
test_period[['predicted_winner_right','vegas_favored_wins','ats_right','season']].groupby("season").agg([np.mean, np.var])

predicted_winner_right           vegas_favored_wins            \
                         mean       var               mean       var   
season                                                                 
2012                 0.607955  0.239708           0.647727  0.229481   
2013                 0.636364  0.232727           0.676136  0.220227   
2014                 0.681818  0.218182           0.681818  0.218182   
2015                 0.659091  0.225974           0.647727  0.229481   
2016                 0.636364  0.232727           0.642045  0.231136   

       ats_right            
            mean       var  
season                      
2012    0.545455  0.249351  
2013    0.539773  0.249838  
2014    0.482955  0.251136  
2015    0.437500  0.247500  
2016    0.500000  0.251429

### Results
ATS is chancey but performance against vegas for winners is pretty close.

In [50]:
# for all years in our sample we do about just as good as vegas but no better
test_period[['predicted_winner_right','vegas_favored_wins','ats_right']].groupby(lambda x: 0).agg([np.mean, np.var])

predicted_winner_right           vegas_favored_wins           ats_right  \
                    mean       var               mean       var      mean   
0               0.644318  0.229433           0.659091  0.224946  0.501136   

             
        var  
0  0.250283

### Next Steps
So now you've seen how to build a strength of schedule (SOS) model. You might think you can make it better, and you probably can, but it would be a better use of your team to create orthogonal features not based on SOS. Something like how a team plays using:  
- offensive scheme: e.g. vertical "Air Coryell" offense
- base_defense: e.g. 3-4
- running yards (and if they are a great running team)
  
Check out our team_season_log table and our team_game_logs to get the data you need to build it.

In [53]:
team_season_log = api.get_dataframe("team_season_log")
team_season_log.head()

,coaches,league_name,losses,playoff_result,points,points_diff,points_opp,rank_def_pts,rank_def_yds,rank_off_pts,rank_off_yds,rank_points_diff,rank_takeaway_giveaway,rank_yds_diff,historical_team_name,team_name,teams_in_league,ties,wins,season,defensive_coordinator,offensive_coordinator,offensive_scheme,base_defense,stadium
0,Arians,NFL,8,,418,56,362,14,2,6,9,7,17,1,Arizona Cardinals,ARI,32,1,7,2016,James Bettcher,Harold Goodwin,Air Coryell,3-4,University of Phoenix
1,Quinn,NFL,5,Lost SB,540,134,406,27,25,1,2,2,4,4,Atlanta Falcons,ATL,32,0,11,2016,Richard Smith,Kyle Shanahan,West Coast,4-3,Georgia Dome
2,Harbaugh,NFL,8,,343,22,321,9,7,21,17,13,9,8,Baltimore Ravens,BAL,32,0,8,2016,Dean Pees,"Marty Mornhinweg, Marc Trestman",West Coast,3-4,M&T; Bank
3,"Ryan,Lynn",NFL,9,,399,21,378,16,19,10,16,14,7,20,Buffalo Bills,BUF,32,0,7,2016,Dennis Thurman,"Anthony Lynn, Greg Roman",West Coast,3-4,New Era Field
4,Rivera,NFL,10,,369,-33,402,26,21,15,19,25,21,25,Carolina Panthers,CAR,32,0,6,2016,Sean McDermott,Mike Shula,Erhardt-Perkins,4-3,Bank of America


In [54]:
team_game_logs = api.get_dataframe("team_game_logs")
team_game_logs.head()

,season,team_name,full_game_type,opp_first_downs,team_first_downs,game_location,game_outcome,game_time,opp_pass_yds,team_pass_yds,opp_pts,team_pts,opp_rush_yds,team_rush_yds,opp_to,team_to,opp_yards,team_yards,game_type,week_num,opp_name,had_overtime,game_datetime
0,2016,DET,Wild Card,24.0,13.0,@,L,8:15PM ET,210.0,182.0,26.0,6.0,177.0,49.0,NaN,NaN,387.0,231.0,Playoffs,1,SEA,False,2017-01-07 20:15:00
1,2016,GB,Wild Card,15.0,23.0,None,W,4:40PM ET,295.0,331.0,13.0,38.0,70.0,75.0,2.0,NaN,365.0,406.0,Playoffs,1,NYG,False,2017-01-08 16:40:00
2,2016,HOU,Wild Card,16.0,19.0,None,W,4:35PM ET,139.0,168.0,14.0,27.0,64.0,123.0,3.0,NaN,203.0,291.0,Playoffs,1,OAK,False,2017-01-07 16:35:00
3,2016,MIA,Wild Card,19.0,18.0,@,L,1:05PM ET,188.0,253.0,30.0,12.0,179.0,52.0,2.0,3.0,367.0,305.0,Playoffs,1,PIT,False,2017-01-08 13:05:00
4,2016,NYG,Wild Card,23.0,15.0,@,L,4:40PM ET,331.0,295.0,38.0,13.0,75.0,70.0,NaN,2.0,406.0,365.0,Playoffs,1,GB,False,2017-01-08 16:40:00
